In [1]:
import notebook_importer

In [2]:
import spdz
import random
import numpy as np
import zmq

importing notebook from spdz.ipynb


In [3]:
# Bob is party 1
party = 1

# Connect to zmq
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")

#TODO: tmp solution remove ASAP
spdz.spdz_socket = socket
spdz.socket_party = party

In [4]:
# Input dataset
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

# Output dataset
y = np.array([[0,0,1,1]]).T

# Split input into shares 
#X_shares = spdz.share_vec(spdz.wrap(X))
X_alice, X_bob = spdz.secure(X)

# Split output into shares
y_alice, y_bob = spdz.secure(y)

In [5]:
# Bob receives initial weights from Alice
syn0_bob = spdz.swap_shares(np.array("OK"), party, socket)

# Bob sends X and y to Alice
print(spdz.swap_shares(X_alice, party, socket))
print(spdz.swap_shares(y_alice, party, socket))

OK
OK


In [6]:
# Multiplication Test
a, b = spdz.PrivateValue.secure(3)
t = spdz.swap_shares(a, party, socket)
res = t * b
other = spdz.swap_shares(res, party, socket)
print(spdz.decode_vec(other + res))

15.0


In [7]:
def np_sigmoid(x):
   return 1 / (1 + np.exp(-x))

sigmoid = spdz.SigmoidInterpolated10(party, socket)

#Sigmoid test:
result = sigmoid.evaluate(X_bob)
result_alice = spdz.swap_shares(result, party, socket)

print("sigmoid result: ")
print(spdz.decode_vec(result_alice + result))

print("np_sigmoid result: ")
print(np_sigmoid(X))

sigmoid result: 
[[ 0.5        0.5        0.7078829]
 [ 0.5        0.7078829  0.7078829]
 [ 0.7078829  0.5        0.7078829]
 [ 0.7078829  0.7078829  0.7078829]]
np_sigmoid result: 
[[ 0.5         0.5         0.73105858]
 [ 0.5         0.73105858  0.73105858]
 [ 0.73105858  0.5         0.73105858]
 [ 0.73105858  0.73105858  0.73105858]]


In [8]:
# Train in sync with Alice

network = spdz.TwoLayerNetwork(sigmoid, party, socket)
network.train(X_bob, y_bob, syn0_bob)

weights = network.print_weights()

print("predictions: ")
preds = network.predict(X_bob)
preds_alice = spdz.swap_shares(preds, party, socket)
print(spdz.decode_vec(preds_alice + preds))

Layer 0 weights: 
[[ 4.7238776]
 [-0.2363572]
 [-2.1361938]]
predictions: 
[[ 0.1111069]
 [ 0.0845082]
 [ 0.9361187]
 [ 0.9132718]]
